In [ ]:
!pip install fastapi uvicorn requests python-dateutil nest_asyncio pyngrok -q


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
NYT_API_KEY = os.getenv("NYT_API_KEY")


In [ ]:
from dateutil.parser import isoparse

DATE_STR = "2002-08-15"   # Change this to any YYYY-MM-DD you want
target_date = isoparse(DATE_STR).date()

print("Using date:", target_date)


In [ ]:
import requests

year = target_date.year
month = target_date.month
url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json"

params = {"api-key": NYT_API_KEY}
response = requests.get(url, params=params, timeout=30)

print("HTTP Status:", response.status_code)


In [ ]:
data = response.json()
docs = data["response"]["docs"]

count = 0
for d in docs:
    pub_date = d["pub_date"][:10]   # first 10 chars = YYYY-MM-DD
    if pub_date == DATE_STR:
        count += 1

print("Articles on this day:", count)


In [ ]:
for d in docs:
    pub_date = d["pub_date"][:10]
    if pub_date == DATE_STR:
        print("-", d["headline"]["main"])


In [ ]:
# Scoring of headlines

section_points = {
    "World": 3, "U.S.": 3,
    "Business": 2, "Technology": 2, "Science": 2, "Health": 2,
    "Sports": 1, "Arts": 1
}

def score(article):
    points = section_points.get(article.get("section_name"), 0)
    if article.get("type_of_material") in {"News", "Front Page"}:
        points += 2
    return points

# Filter for the chosen date
articles_for_day = [a for a in docs if a.get("pub_date", "")[:10] == DATE_STR]

# Sort by score
articles_for_day.sort(key=score, reverse=True)

# Print top 10
print(f"Top 10 NYT headlines for {DATE_STR}:\n")
for i, a in enumerate(articles_for_day[:10], start=1):
    print(f"{i}. {a['headline']['main']}  [{a.get('section_name')}]")
    print(f"   {a['web_url']}\n")


In [ ]:
import csv
from IPython.display import FileLink

# Save CSV
filename = f"top10_{DATE_STR}.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Rank", "Headline", "Section", "URL"])
    for i, a in enumerate(articles_for_day[:10], start=1):
        writer.writerow([i, a["headline"]["main"], a.get("section_name"), a["web_url"]])

# Download link
FileLink(filename)


In [ ]:
# --- Part 2: REST API with FastAPI ---

from fastapi import FastAPI, Query
import nest_asyncio, uvicorn

nest_asyncio.apply()  # Run FastAPI inside Jupyter

app = FastAPI()

@app.get("/headlines")
def headlines(
    date: str = Query(..., description="YYYY-MM-DD"),
    limit: int = Query(10, ge=1, le=50, description="Number of headlines")
):
    articles_for_day = [a for a in docs if a.get("pub_date", "")[:10] == date]
    articles_for_day.sort(key=score, reverse=True)
    top_articles = articles_for_day[:limit]

    return {
        "date": date,
        "count": len(top_articles),
        "headlines": [
            {"headline": a["headline"]["main"],
             "section": a.get("section_name"),
             "url": a["web_url"]}
            for a in top_articles
        ]
    }


In [ ]:
# Start the FastAPI server on your computer at http://127.0.0.1:8000
uvicorn.run(app, host="127.0.0.1", port=8000)